## Laboratoria 3.1. Implementacja algorytmu genetycznego – opt. f-cji kwadratowej w R³
## [Zadanie](https://pages.mini.pw.edu.pl/~karwowskij/mioad/lab-gen.html)

Napisać podstawowy algorytm genetyczny z mutacją gaussowską i krzyżowaniem jednopunktowym.  
Sprawdzić działanie algorytmu na funkcji $x^2+y^2+2z^2$ oraz na pięciowymiarowej funkcji Rastrigina. 

In [1]:
import math
import numpy as np
import random
import pandas as pd

# objective functions
class Multinomial:
    @staticmethod
    def calculate(population: np.array):
        y = []
        for individual in population:
            y.append(individual[0]**2+individual[1]**2+2*individual[2]**2)
        return y
    
class Rastrigin:
    @staticmethod
    def generate_cube(size = 5, dim = 5):
        return np.random.uniform(-5.12, 5.12, size=(size, dim))
    def calculate(cube):
        result = []
        for row in cube:
            res = 10*cube.shape[1] + np.sum(np.power(row, 2) - 10*np.cos(2*math.pi*row))
            result.append(res)
        return result

In [2]:
class GeneticAlgorithm:
    def __init__(self, objective_class, data_dimension, population_size):
        self.objective_class = objective_class
        self.data_dimension = data_dimension
        self.population_size = population_size
    
    def generate_data(self):
        if self.objective_class == Multinomial:
            x = np.random.uniform(-1, 1, size=(self.population_size, self.data_dimension))
            y = self.objective_class.calculate(x)
        elif self.objective_class == Rastrigin:
            x = self.objective_class.generate_cube(self.population_size, self.data_dimension)
            y = self.objective_class.calculate(x)
        return x, y
    
    def RouletteSelection(self, population, scores,  k=2):
        """Select *k* individuals from the input *population* using *k*
        spins of a roulette. The selection is made by looking only at the first
        objective of each individual. The list returned contains references to
        the input *individuals*.
        Parameters:
            individuals: A list of individuals to select from.
            k: The number of individuals to select.
        Returns: A list of selected individuals.


        The roulette selection by definition cannot be used for minimization
        or when the fitness can be smaller or equal to 0.
        """

        scores_total = np.sum(scores)
        ind_probabilities = (scores/scores_total).astype('float64')
        
        idxs_arr = np.array(range(0, population.shape[0], 1))
        chosen_idxs = np.random.choice(a = idxs_arr, size = k, p = ind_probabilities)
        return population[chosen_idxs]
    
    def BestSelection(self, population, scores, k=2):
        """
        Select the k best individuals among the input individuals.
        """
        sorting_permutation = sorted(range(len(scores)), key=lambda k: scores[k], reverse=False)
        return population[sorting_permutation[:k]]

    # crossover functon
    def OnePointCrossover(self, ind1, ind2, crossover_probability = 0.8):
        """Executes a one point crossover on the input individuals.
        The two individuals are modified in place. The resulting individuals will
        respectively have the length of the other.
        Parameters:
            ind1: The first individual participating in the crossover.
            ind2: The second individual participating in the crossover.
        Returns:
            A tuple of two individuals.
        """
        child1, child2 = ind1.copy(), ind2.copy()
        size = min(len(ind1), len(ind2))
        if random.uniform(0, 1) >= crossover_probability:
            crossover_point = random.randint(1, size - 1)
            child1[crossover_point:] = ind1[:crossover_point] + ind2[crossover_point:]
            child2[crossover_point:] = ind2[:crossover_point] + ind1[crossover_point:]
        return [child1, child2]
    
    # mutation function
    def mutGaussian(self, individual, mean=0, stdev=1, mutation_probability = 0.2):
        """This function applies a gaussian mutation of mean mu and standard
        deviation sigma on the input individual.
        This mutation expects parameters listed below:
            individuals - individuals to be mutated, `sequence` composed of real valued attributes.
            indproba - argument which is the independent probability of each attribute to be mutated.
            mu - mean or sequence of means for the gaussian addition mutation.
            stdev: Standard deviation or python sequence of standard deviations for the gaussian addition mutation.
        Returns: A tuple of one individual.
        """
        size = len(individual)
        if not isinstance(mean, np.ndarray):
            mean = np.repeat(mean, size)
        elif len(mean) < size:
            raise IndexError("Mean must be at least the size of individual: %d < %d" % (len(mean), size))
            
        if not isinstance(stdev, np.ndarray):
            stdev = np.repeat(stdev, size)
        elif len(stdev) < size:
            raise IndexError("Standard deviation must be at least the size of individual: %d < %d" % (len(stdev), size))

        for i, m, s in zip(range(size), mean, stdev):
            if random.uniform(0, 1) <= mutation_probability:
                individual[i] += random.gauss(m, s)
        return individual
    
    def evaluate(self, n_generations, selection_func, crossover_probability=0.8, mutation_probability=0.2):
        population, scores = self.generate_data()
        # best solution initialization
        best, best_eval = population[0], scores[0]    
        # best solutions storage
        generation_num = [] 
        
        for i in range(n_generations):
            # check for new best solution
            for j in range(population.shape[0]):
                if scores[j] < best_eval:
                    best, best_eval = population[j], scores[j]
                    print("> Generation %d, new best f(%s) = %f" % (i+1,  best, best_eval))
                    generation_num.append(i+1)
            # parents selection
            selected = [selection_func(population, scores) for i in range(population.shape[0])]
            
            # children creation
            children = []
            
            for k in range(0, population.shape[0], 2):
               # get selected parents in pairs
                p1, p2 = selected[k], selected[k+1]
                # crossover and mutation
                for c in self.OnePointCrossover(p1, p2, crossover_probability)[0]:
                    # mutation
                    mutated = self.mutGaussian(c, mutation_probability)
                    # store for next generation
                    children.append(mutated)
                        
            # replace population
            children_array = np.array([list(arr) for arr in children])
            population = children_array
            # calculate new scores
            scores = self.objective_class.calculate(population)
        print("Optimization completed!")    
        return [best, best_eval], generation_num[-1]

## Sprawdzenia działania algorytmu na funkcji $f(x,y,z) = x^2+y^2+2z^2$
#### Faktyczne minimum tej funkcji: $f(0,0,...,0) = 0$

In [3]:
GeneticAlgorithm_Multinomial = GeneticAlgorithm(Multinomial, 3, 30)

In [4]:
# Roulette selection function
optimum, n_generations = GeneticAlgorithm_Multinomial.evaluate(selection_func=GeneticAlgorithm_Multinomial.RouletteSelection, n_generations=100)

> Generation 1, new best f([ 0.90443728  0.29473538 -0.00957689]) = 0.905059
> Generation 1, new best f([-0.69437957 -0.40808941  0.03194396]) = 0.650741
> Generation 1, new best f([-0.47924406 -0.25532536 -0.36909155]) = 0.567323
> Generation 1, new best f([ 0.15844641 -0.67918772 -0.07035524]) = 0.496301
> Generation 1, new best f([0.18135475 0.58354253 0.07157184]) = 0.383656
> Generation 1, new best f([ 0.01520972  0.13594225 -0.26894917]) = 0.163379
Optimization completed!


In [5]:
# Best selection function
optimum, n_generations = GeneticAlgorithm_Multinomial.evaluate(selection_func=GeneticAlgorithm_Multinomial.BestSelection, n_generations=100)

> Generation 1, new best f([ 0.9849851  0.4351252 -0.0293603]) = 1.161254
> Generation 1, new best f([-0.94051418 -0.02201274  0.01153068]) = 0.885317
> Generation 1, new best f([-0.23643682 -0.03494226 -0.45787247]) = 0.476418
> Generation 1, new best f([-0.42180232 -0.25409821 -0.03730994]) = 0.245267
> Generation 2, new best f([-0.22484708 -0.25409821 -0.03730994]) = 0.117906
> Generation 3, new best f([-0.11786468 -0.25409821 -0.03730994]) = 0.081242
> Generation 3, new best f([-0.22484708  0.08406667 -0.03730994]) = 0.060407
> Generation 4, new best f([-0.22484708  0.08406667 -0.01271692]) = 0.057947
> Generation 5, new best f([-0.06178075  0.08406667 -0.03730994]) = 0.013668
> Generation 9, new best f([ 0.0478421   0.08406667 -0.03730994]) = 0.012140
> Generation 10, new best f([-0.04070645  0.08406667 -0.03730994]) = 0.011508
> Generation 11, new best f([-0.03843325  0.08406667 -0.03730994]) = 0.011328
> Generation 11, new best f([-0.04070645  0.08406667 -0.00909272]) = 0.008890

### Wywołanie w pętli na potrzeby sprawozdania

In [8]:
roulette_opt_values = []
roulette_gen_numbers = []
ranking_opt_values = []
ranking_gen_numbers = []

for i in range(10):
    # functions call
    GeneticAlgorithm_Multinomial = GeneticAlgorithm(Multinomial, 3, 30)
    roulette_optimum, roulette_generations = GeneticAlgorithm_Multinomial.evaluate(selection_func=GeneticAlgorithm_Multinomial.RouletteSelection, n_generations=100)
    GeneticAlgorithm_Multinomial = GeneticAlgorithm(Multinomial, 3, 30)
    best_optimum, best_generations = GeneticAlgorithm_Multinomial.evaluate(selection_func=GeneticAlgorithm_Multinomial.BestSelection, n_generations=100)
    
    roulette_opt_values.append(roulette_optimum[1])
    roulette_gen_numbers.append(roulette_generations)
    
    ranking_opt_values.append(best_optimum[1])
    ranking_gen_numbers.append(best_generations)

> Generation 1, new best f([-0.20086467  0.40999469 -0.61806043]) = 0.972440
> Generation 1, new best f([-0.32366715  0.39854358 -0.35213527]) = 0.511596
> Generation 1, new best f([-0.16631513 -0.25522602  0.13011304]) = 0.126660
Optimization completed!
> Generation 1, new best f([ 0.68411768 -0.35328893 -0.49815965]) = 1.089156
> Generation 1, new best f([ 0.18346604  0.30109521 -0.02657489]) = 0.125731
> Generation 2, new best f([0.27307067 0.14762932 0.09412148]) = 0.114080
> Generation 2, new best f([ 0.18346604  0.18915299 -0.02657489]) = 0.070851
> Generation 2, new best f([ 0.13106633 -0.16686313  0.06754659]) = 0.054147
> Generation 5, new best f([ 0.13106633 -0.04785347  0.06754659]) = 0.028593
> Generation 7, new best f([ 0.04121849 -0.04785347  0.06754659]) = 0.013114
> Generation 8, new best f([-0.0240688  -0.04785347  0.06754659]) = 0.011994
> Generation 9, new best f([-0.01067036 -0.04785347  0.06754659]) = 0.011529
> Generation 10, new best f([-0.03473916 -0.09570695  0

Optimization completed!
> Generation 1, new best f([ 0.16878796 -0.81935743  0.75797637]) = 1.848892
> Generation 1, new best f([-0.72649179  0.90486603  0.09497939]) = 1.364615
> Generation 1, new best f([-0.3249462  -0.86807817 -0.43810826]) = 1.243027
> Generation 1, new best f([-0.09325703  0.77800567  0.25673   ]) = 0.745810
> Generation 1, new best f([-0.70010629 -0.15734803 -0.21553285]) = 0.607816
> Generation 2, new best f([-0.44760716  0.53643433 -0.00617635]) = 0.488190
> Generation 2, new best f([0.25249914 0.4031537  0.2093565 ]) = 0.313949
> Generation 2, new best f([-0.44760716  0.29680407 -0.00617635]) = 0.288521
> Generation 3, new best f([-0.44760716  0.11620184 -0.00617635]) = 0.213931
> Generation 3, new best f([-0.19510802  0.02381981  0.20318014]) = 0.121199
> Generation 4, new best f([0.07703872 0.02381981 0.20318014]) = 0.089067
> Generation 5, new best f([ 0.07703872  0.02381981 -0.17255817]) = 0.066055
> Generation 6, new best f([0.15407745 0.04763962 0.030621

In [9]:
print('Target function: mutinomial')
pd.DataFrame({'Selection function': ['Roulette', 'Ranking'],
              'Min target function value': [np.min(roulette_opt_values), np.min(ranking_opt_values)],
              'Mean target function value': [np.mean(roulette_opt_values), np.mean(ranking_opt_values)],
              'Max target function value': [np.max(roulette_opt_values), np.max(ranking_opt_values)],
              'Target function value std dev': [np.std(roulette_opt_values), np.std(ranking_opt_values)],
              'Min number of generations': [np.min(roulette_gen_numbers), np.min(ranking_gen_numbers)],
              'Mean number of generations': [np.mean(roulette_gen_numbers), np.mean(ranking_gen_numbers)],
              'Max number of generations': [np.max(roulette_gen_numbers), np.max(ranking_gen_numbers)],
              'Number of generations std dev': [np.std(roulette_gen_numbers), np.std(ranking_gen_numbers)]})

Target function: mutinomial


,Selection function,Min target function value,Mean target function value,Max target function value,Target function value std dev,Min number of generations,Mean number of generations,Max number of generations,Number of generations std dev
0,Roulette,0.004344,0.077788,0.133736,0.044211,1,1.6,4,0.916515
1,Ranking,0.000009,0.000061,0.000231,0.000060,55,80.5,99,12.579746


## Sprawdzenia działania algorytmu na pięciowymiarowej funkcji Rastrigina $$f(x_{1},...,x_{d}) = 10d + \sum_{i=1}^d (x_{i}^2-10cos(2\pi x_{i})),$$ gdzie $-5.12 \leq x_{i} \leq 5.12$.
#### Faktyczne minimum tej funkcji: $f(0,0,...,0) = 0$

In [10]:
GeneticAlgorithm_Rastrigin = GeneticAlgorithm(Rastrigin, 5, 30)

In [11]:
optimum, n_generations = GeneticAlgorithm_Rastrigin.evaluate(selection_func=GeneticAlgorithm_Rastrigin.RouletteSelection, n_generations=150)

> Generation 1, new best f([ 0.19712521  1.28221264 -3.90391334 -2.24806069  1.11307156]) = 56.030146
> Generation 1, new best f([ 2.3945863   1.95852385  1.96197498 -3.80809829  0.98591424]) = 53.869392
> Generation 1, new best f([-2.05640823  3.94748025  1.26810203  0.97332181  1.02337101]) = 35.957938
Optimization completed!


In [12]:
optimum, n_generations = GeneticAlgorithm_Rastrigin.evaluate(selection_func=GeneticAlgorithm_Rastrigin.BestSelection, n_generations=150)

> Generation 1, new best f([-1.41529358  2.75839979  2.18983087 -5.05422226  1.91636188]) = 79.947020
> Generation 1, new best f([-1.12804405 -3.87226208  1.9403326   3.44976802  0.12188748]) = 61.057240
> Generation 1, new best f([ 0.7931208   0.15775873 -0.67316058  2.86612557 -0.31762688]) = 53.368652
> Generation 2, new best f([ 0.7931208   0.15775873 -0.67316058  2.86612557 -0.21653542]) = 47.105062
> Generation 2, new best f([ 0.7931208   0.15775873 -0.48106738  2.86612557 -0.10288299]) = 46.238600
> Generation 2, new best f([ 0.04053023  0.15775873 -0.67316058  1.96506427 -0.31762688]) = 38.292851
> Generation 3, new best f([ 0.04053023  0.15775873 -0.67316058  1.96506427  1.06226759]) = 25.953595
> Generation 3, new best f([ 0.04053023  0.15775873 -0.67316058  1.96506427 -0.02719708]) = 24.215948
> Generation 4, new best f([ 0.04053023 -0.87092103 -0.67316058  1.96506427 -0.02719708]) = 23.538656
> Generation 4, new best f([ 0.04053023  0.15775873 -1.07234345  1.96506427  1.131

### Wywołanie w pętli na potrzeby sprawozdania

In [14]:
roulette_opt_values = []
roulette_gen_numbers = []
ranking_opt_values = []
ranking_gen_numbers = []

for i in range(10):
    # functions call
    GeneticAlgorithm_Rastrigin = GeneticAlgorithm(Rastrigin, 5, 30)
    roulette_optimum, roulette_generations = GeneticAlgorithm_Rastrigin.evaluate(selection_func=GeneticAlgorithm_Rastrigin.RouletteSelection, n_generations=150)
    GeneticAlgorithm_Rastrigin = GeneticAlgorithm(Rastrigin, 5, 30)
    best_optimum, best_generations = GeneticAlgorithm_Rastrigin.evaluate(selection_func=GeneticAlgorithm_Rastrigin.BestSelection, n_generations=150)
    
    roulette_opt_values.append(roulette_optimum[1])
    roulette_gen_numbers.append(roulette_generations)
    
    ranking_opt_values.append(best_optimum[1])
    ranking_gen_numbers.append(best_generations)

> Generation 1, new best f([-4.97292219 -1.21194783  4.2264503  -1.98549649 -1.52139381]) = 86.571702
> Generation 1, new best f([ 1.11734694 -1.14729066 -5.0266242  -2.98250417 -1.37948759]) = 62.680079
> Generation 1, new best f([ 0.87736147 -1.58878086 -1.06500014  0.25108796  0.22156123]) = 44.962668
> Generation 1, new best f([-1.73712335  3.81229054  0.05673752  0.9681246   0.86832415]) = 40.299424
Optimization completed!
> Generation 1, new best f([ 0.20010322  1.36717677 -2.9189256  -1.52828678 -2.5055604 ]) = 83.779948
> Generation 1, new best f([ 4.87911396 -0.9923771   2.63334444  1.20804529 -2.78581917]) = 75.558720
> Generation 1, new best f([-0.86709715 -4.04294906  3.03614071  1.12172781  0.48086337]) = 54.425050
> Generation 1, new best f([-1.00516299  0.94642766  3.5931777   1.09037977  2.10499808]) = 43.005922
> Generation 2, new best f([-0.86709715 -4.04294906  3.03614071  1.12172781 -0.10932351]) = 36.545816
> Generation 3, new best f([-1.00516299  0.94642766  3.858

> Generation 146, new best f([ 0.00419751 -0.00064524  0.00129284 -0.00374228 -0.0003378 ]) = 0.006710
Optimization completed!
> Generation 1, new best f([ 1.94378138 -3.84071767  1.27829498 -1.04841198  0.77526452]) = 47.731550
> Generation 1, new best f([ 2.04401805  3.32876031 -0.84521477 -0.76349519  0.93623667]) = 46.874644
> Generation 1, new best f([ 0.02149882 -0.27444646  0.62843417  2.11763359 -1.02928136]) = 37.330549
Optimization completed!
> Generation 1, new best f([ 2.83715793 -4.72394457 -0.07165152 -3.32185751  0.90371835]) = 75.779083
> Generation 1, new best f([ 0.85398655 -0.8999198  -3.3352844   3.76753864 -1.99133525]) = 60.678042
> Generation 1, new best f([-0.1712963   2.23571486 -2.09035535 -0.06424349 -2.56279725]) = 51.931062
> Generation 1, new best f([ 4.01814921  0.139934    2.90270215 -0.28362872 -1.94147045]) = 46.705146
> Generation 2, new best f([ 4.01814921  0.139934    2.90270215 -0.02081974 -1.94147045]) = 34.613307
> Generation 3, new best f([ 2.02

Optimization completed!
> Generation 1, new best f([ 1.83588774  1.41285842  0.41469328 -1.04970042  1.96510469]) = 53.222561
Optimization completed!
> Generation 1, new best f([ 0.52865834 -1.63201586  1.78338799  3.09469523  0.18643205]) = 68.073465
> Generation 1, new best f([ 0.66312216  3.83534785  2.01744527 -1.95778387 -0.58056455]) = 62.627754
> Generation 1, new best f([-2.31439323 -1.7576492  -0.90650529  1.05495712  1.12340724]) = 40.223716
> Generation 2, new best f([-2.31439323 -1.7576492  -0.91174674  1.05495712  1.12340724]) = 40.055266
> Generation 2, new best f([-0.88367891 -1.7576492  -1.10057041  2.1519742   1.12340724]) = 32.060544
> Generation 3, new best f([-1.09708017 -1.7576492  -0.90650529  1.05495712  1.12340724]) = 23.937787
> Generation 5, new best f([-1.09708017 -3.00087146 -0.90650529  1.05495712  1.12340724]) = 20.334264
> Generation 5, new best f([-1.09708017 -0.90492115 -0.90650529  1.05495712  1.12340724]) = 13.879747
> Generation 6, new best f([ 0.038

> Generation 125, new best f([-0.00859684  0.00208254  0.00343314  0.0085218   0.00831941]) = 0.045990
> Generation 129, new best f([-0.00761903  0.00208254  0.00343314  0.0085218   0.00831941]) = 0.042845
Optimization completed!
> Generation 1, new best f([-2.73988749  4.94597383  0.17216846 -2.71009855  0.38200717]) = 85.854087
> Generation 1, new best f([ 3.58876422  1.13221328 -0.24355518  3.21506328 -3.00569142]) = 72.756470
> Generation 1, new best f([-1.66332502 -0.06572252 -4.30190165  1.03336779  0.01140819]) = 51.814838
Optimization completed!
> Generation 1, new best f([-3.89995923 -2.81568768 -0.91411026 -1.32547311 -4.8086443 ]) = 79.139185
> Generation 1, new best f([ 0.54157016  2.75833781  1.61894617  3.18531185 -1.0587517 ]) = 74.982077
> Generation 1, new best f([-2.73797433  2.75702622  0.8671077  -2.8381436   3.2192383 ]) = 70.689578
> Generation 1, new best f([-2.24752500e+00 -2.52533916e+00 -2.22920870e+00  2.03049371e+00
  5.07502572e-04]) = 59.119402
> Generatio

> Generation 144, new best f([0.01692051 0.00079092 0.00015454 0.00192065 0.00237178]) = 0.058724
> Generation 145, new best f([0.00432425 0.00079092 0.00015454 0.00192065 0.00237178]) = 0.005686
Optimization completed!


In [15]:
print('Target function: Rastrigin')
pd.DataFrame({'Selection function': ['Roulette', 'Ranking'],
              'Min target function value': [np.min(roulette_opt_values), np.min(ranking_opt_values)],
              'Mean target function value': [np.mean(roulette_opt_values), np.mean(ranking_opt_values)],
              'Max target function value': [np.max(roulette_opt_values), np.max(ranking_opt_values)],
              'Target function value std dev': [np.std(roulette_opt_values), np.std(ranking_opt_values)],
              'Min number of generations': [np.min(roulette_gen_numbers), np.min(ranking_gen_numbers)],
              'Mean number of generations': [np.mean(roulette_gen_numbers), np.mean(ranking_gen_numbers)],
              'Max number of generations': [np.max(roulette_gen_numbers), np.max(ranking_gen_numbers)],
              'Number of generations std dev': [np.std(roulette_gen_numbers), np.std(ranking_gen_numbers)]})

Target function: Rastrigin


,Selection function,Min target function value,Mean target function value,Max target function value,Target function value std dev,Min number of generations,Mean number of generations,Max number of generations,Number of generations std dev
0,Roulette,33.893293,45.131318,54.783526,7.378357,1,2.0,6,1.673320
1,Ranking,0.002637,0.036207,0.112761,0.037088,111,133.4,147,12.240915


### Wniosek: Funkcja selekcji Roulette nie sprawdza się dla problemu minimalizacji.